In [25]:
# Import required libraries

# Data Handling
from __future__ import division
import pandas as pd
pd.options.display.max_rows = 999
import numpy as np
import datetime as dt

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Data Access
from google.cloud import bigquery
client = bigquery.Client()
%load_ext google.cloud.bigquery

/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cl

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## 1. Business Understanding

As a digital analytics enthusiast, it would be quite interesting to apply my data analysis skills here.
<br></br>
My key interests would be:
1. What percentage of users is actually buying and what is there average spend?
2. How are users and revenue distributed geographically?
3. Can a user's revenue be predicted taking into account his previous user journey?
<br></br>

## 2. Data Understanding

In [41]:
%%bigquery df
# Query the data from BigQuery public dataset

#standardSQL
WITH query_settings AS (
  SELECT
    '20170501' AS start_date,
    '20170801' AS end_date), 

session_level AS (
SELECT 
fullVisitorId, 
CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS session_id,
date,
visitNumber,
visitStartTime,
device.deviceCategory AS device,
geoNetwork.country AS country,
socialEngagementType,
totals.hits AS hits,
totals.pageviews AS pageviews,
IFNULL(totals.timeOnSite, 0) AS time_on_site,
IFNULL(totals.bounces, 0) AS bounces,
IFNULL(totals.transactions, 0) AS transactions,
IFNULL(totals.totalTransactionRevenue, 0) AS revenue,
totals.newVisits AS new_visitor,
totals.sessionQualityDim AS session_quality,
channelgrouping,
trafficSource.medium AS medium,
trafficSource.source AS source
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, query_settings 
WHERE _TABLE_SUFFIX BETWEEN start_date AND end_date),

hit_level AS (
SELECT session_id_p, MAX(landing_page) AS landing_page, MAX(exit_page) AS exit_page
FROM(
SELECT
CONCAT(fullVisitorId, CAST(visitStartTime AS STRING)) AS session_id_p,
FIRST_VALUE(hits.page.pagePath)
    OVER (PARTITION BY CONCAT(fullVisitorId, CAST(visitStartTime AS STRING)) ORDER BY hits.time ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS landing_page,
LAST_VALUE(hits.page.pagePath)
    OVER (PARTITION BY CONCAT(fullVisitorId, CAST(visitStartTime AS STRING)) ORDER BY hits.time ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS exit_page
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` AS GA, query_settings, UNNEST(GA.hits) AS hits 
WHERE (_TABLE_SUFFIX BETWEEN start_date AND end_date) AND (hits.type='PAGE'))
GROUP BY 1)

SELECT * EXCEPT(session_id_p)
FROM session_level AS sl
LEFT JOIN hit_level AS hl
ON sl.session_id = hl.session_id_p

,fullVisitorId,session_id,date,visitNumber,visitStartTime,device,country,socialEngagementType,hits,pageviews,...,bounces,transactions,revenue,new_visitor,session_quality,channelgrouping,medium,source,landing_page,exit_page
0,9511832364821210514,95118323648212105141494933727,20170516,1,1494933727,desktop,France,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Referral,referral,docs.google.com,/home,/home
1,4823353882333320348,48233538823333203481494964509,20170516,1,1494964509,mobile,Netherlands,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,google,/home,/home
2,0800568178159810103,08005681781598101031494924340,20170516,1,1494924340,desktop,Ukraine,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,google,/home,/home
3,9599884333980345446,95998843339803454461494961588,20170516,1,1494961588,desktop,New Zealand,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,google,/home,/home
4,853164492434479989,8531644924344799891494946519,20170516,1,1494946519,desktop,United States,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,google,/home,/home
5,5855138621999659934,58551386219996599341494975945,20170516,1,1494975945,desktop,United States,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,yahoo,/home,/home
6,9319719567047323806,93197195670473238061494944029,20170516,1,1494944029,desktop,United States,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,google,/home,/home
7,4909720397538871854,49097203975388718541494934147,20170516,1,1494934147,desktop,United Kingdom,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,google,/home,/home
8,4983368006308813795,49833680063088137951494947878,20170516,1,1494947878,desktop,Germany,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,google,/home,/home
9,395624752702261449,3956247527022614491494927483,20170516,1,1494927483,desktop,Netherlands,Not Socially Engaged,1,1.0,...,1,0,0,1.0,NaN,Organic Search,organic,google,/home,/home


In [42]:
# Get the number of sessions.
print("The dataset consists of {} sessions.".format(df.shape[0]))
print("The time period of analysis ranges from {} to {}.".format(df['date'].min(), df['date'].max()))

The dataset consists of 203317 sessions.
The time period of analysis ranges from 20170501 to 20170801.


In [43]:
# Get the data format for each column.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203317 entries, 0 to 203316
Data columns (total 21 columns):
fullVisitorId           203317 non-null object
session_id              203317 non-null object
date                    203317 non-null object
visitNumber             203317 non-null int64
visitStartTime          203317 non-null int64
device                  203317 non-null object
country                 203317 non-null object
socialEngagementType    203317 non-null object
hits                    203317 non-null int64
pageviews               203286 non-null float64
time_on_site            203317 non-null int64
bounces                 203317 non-null int64
transactions            203317 non-null int64
revenue                 203317 non-null int64
new_visitor             153023 non-null float64
session_quality         68379 non-null float64
channelgrouping         203317 non-null object
medium                  203317 non-null object
source                  203317 non-null object
l

In [44]:
# Show what kind of data is provided
df.columns

Index([u'fullVisitorId', u'session_id', u'date', u'visitNumber',
       u'visitStartTime', u'device', u'country', u'socialEngagementType',
       u'hits', u'pageviews', u'time_on_site', u'bounces', u'transactions',
       u'revenue', u'new_visitor', u'session_quality', u'channelgrouping',
       u'medium', u'source', u'landing_page', u'exit_page'],
      dtype='object')

In [45]:
# Look for missing values
df.isna().sum()

fullVisitorId                0
session_id                   0
date                         0
visitNumber                  0
visitStartTime               0
device                       0
country                      0
socialEngagementType         0
hits                         0
pageviews                   31
time_on_site                 0
bounces                      0
transactions                 0
revenue                      0
new_visitor              50294
session_quality         134938
channelgrouping              0
medium                       0
source                       0
landing_page               937
exit_page                  937
dtype: int64

## Data Preprocessing

#### Pageviews

Rows where 'pageviews' contains NULL values, but a landing page is existing, are likely to be "mis"tracked. Therefore, these rows are dropped from the further analysis. 

In [22]:
# Drop rows
data = df[df['pageviews'].isnull()==False]

#### Session Quality

An estimate of how close a particular session was to transacting, ranging from 1 to 100, calculated for each session. A value closer to 1 indicates a low session quality, or far from transacting, while a value closer to 100 indicates a high session quality, or very close to transacting. A value of 0 indicates that Session Quality is not calculated for the selected time range. So, if this value is totally missing it's not be being calculated at all. Since more than half of the observations contain NULL values, it seems appropriate to delete the column as a whole.

In [36]:
# Drop column
data.drop(labels='session_quality', axis=1, inplace=True)

/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


#### New Visitor

BigQuery documentation states: Total number of new users in session (for convenience). If this is the first visit, this value is 1, otherwise it is null. So, NULL values can be easily replaced with 0.

In [37]:
data['new_visitor'].fillna(value=0, inplace = True)

#### Landing Page

In [38]:
data[data['landing_page'].isnull()==True]

,fullVisitorId,session_id,date,visitNumber,visitStartTime,device,country,socialEngagementType,hits,pageviews,time_on_site,bounces,transactions,revenue,new_visitor,channelgrouping,medium,source,landing_page,exit_page
461,81052488948273085,810524889482730851494258969,20170508,1,1494258974,desktop,United States,Not Socially Engaged,5,5.0,1027,0,0,0,1.0,Paid Search,cpc,google,None,None
736,51131212337208622,511312123372086221494288151,20170508,1,1494288191,desktop,Italy,Not Socially Engaged,20,16.0,246,0,0,0,1.0,Direct,(none),(direct),None,None
754,0843788975236519934,08437889752365199341494263767,20170508,1,1494263769,desktop,United States,Not Socially Engaged,23,20.0,1056,0,1,200360000,1.0,Direct,(none),(direct),None,None
1118,5652517274226327641,56525172742263276411494232706,20170508,1,1494232710,desktop,United States,Not Socially Engaged,1,1.0,0,1,0,0,1.0,Direct,(none),(direct),None,None
1153,9682604624886875077,96826046248868750771494271763,20170508,1,1494271767,desktop,United States,Not Socially Engaged,1,1.0,0,1,0,0,1.0,Direct,(none),(direct),None,None
1334,7434235566542148581,74342355665421485811494249938,20170508,1,1494250486,desktop,Germany,Not Socially Engaged,1,1.0,0,1,0,0,1.0,Direct,(none),(direct),None,None
4511,0014347584561790138,00143475845617901381496678973,20170605,1,1496678982,mobile,United States,Not Socially Engaged,1,1.0,0,1,0,0,1.0,Paid Search,cpc,google,None,None
4534,5073543941976946450,50735439419769464501496709534,20170605,1,1496709540,desktop,United States,Not Socially Engaged,1,1.0,0,1,0,0,1.0,Direct,(none),(direct),None,None
4566,0261996821293730070,02619968212937300701496726456,20170605,1,1496726461,desktop,United States,Not Socially Engaged,1,1.0,0,1,0,0,1.0,Direct,(none),(direct),None,None
4577,18765085218832270,187650852188322701496688305,20170605,1,1496688310,mobile,United States,Not Socially Engaged,1,1.0,0,1,0,0,1.0,Direct,(none),(direct),None,None
